### Import Required Libraries and Set Up Environment Variables

In [ ]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [ ]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")
print(tmdb_api_key)
print(nyt_api_key)


### Access the New York Times API

In [ ]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}&fq={filter_query}&sort={sort}&fl={field_list}"


In [ ]:
# Create an empty list to store the reviews
star_review = []

# loop through pages 0-19
for page in range(20):
    query_url = f"{url}fq={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}&page={page}&api-key={nyt_api_key}"
    response = requests.get(query_url)
    reviews = response.json()
    

    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:

        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews["response"]["docs"]:
            star_review.append(review)

        # Print the page that was just retrieved
        print ("page=",page)
    except:        

        # Print the page number that had no results then break from the loop
        print ("page with no results",page)
        break



In [ ]:
# Preview the first 5 results in JSON format
print(json.dumps(star_review[:5],indent=4))
# Use json.dumps with argument indent=4 to format data


In [ ]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(star_review)


In [ ]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
reviews_df["title"] = reviews_df["headline.main"].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])


In [ ]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df["keywords"] = reviews_df["keywords"].apply(extract_keywords)


In [ ]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = reviews_df["title"].tolist()
titles

### Access The Movie Database API

In [ ]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [ ]:
# Create an empty list to store the results


# Create a request counter to sleep the requests after a multiple
# of 50 requests
tmdb_movies_list = []
request_counter = 1


# Loop through the titles
for title in titles:
    query_url = f"{url}{title}&api_key={tmdb_api_key}"
    response = requests.get(query_url)
    movie_data = response.json()

    # Check if we need to sleep before making a request


    # Add 1 to the request counter
request_counter += 1
if request_counter % 50 == 0:
        print("Sleeping for a second...")
        time.sleep(1)

    
    # Perform a "GET" request for The Movie Database


    # Include a try clause to search for the full movie details.
        
try:
        movie_id = movie_data["results"][0]["id"]
        movie_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
        movie_details = requests.get(movie_url).json()
        
        genres = [genre['name'] for genre in movie_details['genres']]
        spoken_languages = [lang['english_name'] for lang in movie_details['spoken_languages']]
        production_countries = [country['name'] for country in movie_details['production_countries']]
        
        movie_dict = {
            "title": title,
            "original_title": movie_details.get('original_title', ''),
            "budget": movie_details.get('budget', 0),
            "original_language": movie_details.get('original_language', ''),
            "homepage": movie_details.get('homepage', ''),
            "overview": movie_details.get('overview', ''),
            "popularity": movie_details.get('popularity', 0),
            "runtime": movie_details.get('runtime', 0),
            "revenue": movie_details.get('revenue', 0),
            "release_date": movie_details.get('release_date', ''),
            "vote_average": movie_details.get('vote_average', 0),
            "vote_count": movie_details.get('vote_count', 0),
            "genres": genres,
            "spoken_languages": spoken_languages,
            "production_countries": production_countries}
   


        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
    
        tmdb_movies_list.append(movie_dict)
        print(f"Movie '{title}' found and added.")
    
except IndexError:
        print(f"Movie '{title}' not found.")

        
        # Print out the title that was found



In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))



In [ ]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)

tmdb_df.head()


### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(reviews_df, tmdb_df, on="title")

In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
columns_to_fix = ["genres", "spoken_languages", "production_countries"]
characters_to_remove = ["[", "]", "'"]

# Create a list of the columns that need fixing


# Create a list of characters to remove


# Loop through the list of columns to fix
for col in columns_to_fix:
    merged_df[col] = merged_df[col].astype(str)
    for char in characters_to_remove:
        merged_df[col] = merged_df[col].str.replace(char, '')

merged_df.head()


    # Convert the column to type 'str'


    # Loop through characters to remove


# Display the fixed DataFrame


In [ ]:
# Drop "byline.person" column
merged_df = merged_df.drop(columns=["byline.person"], errors="ignore").drop_duplicates()

In [ ]:
# Delete duplicate rows and reset index
merged_df.to_csv("merged_movie_data.csv", index=False)


In [ ]:
# Export data to CSV without the index
